In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('data/data_total.csv')

In [4]:
df.isnull().sum()
#1. 바코드 nan --> 알수없음
#2. capacity nan --> 알수없음
#3, seller --> 알수없음
#4. 영양 --> 알수 없음
#5. prdkindstate --> 알수 없음
#6. prdkind --> 알수없음
#allergy 19 null--> 알수없음


rnum                0
prdlstReportNo      0
productGb           0
prdlstNm            0
rawmtrl             0
allergy            19
prdkind             1
prdkindstate        2
manufacture         0
imgurl1             0
imgurl2             0
nutrient          300
seller            194
barcode           321
capacity          367
dtype: int64

In [5]:
df_allergy = df[['rnum','rawmtrl','allergy']]
df_allergy = pd.DataFrame(df_allergy)
df_allergy = df_allergy.fillna('알수없음')

# 전처리

## 1. 맞춤법

In [6]:
#치명적 오탈자 교체
df_allergy['allergy'] = df_allergy['allergy'].str.replace('복훙아', '복숭아')
df_allergy['allergy'] = df_allergy['allergy'].str.replace('함유식품', '')
df_allergy['allergy'] = df_allergy['allergy'].str.replace('식품', '')
df_allergy['allergy'] = df_allergy['allergy'].str.replace(' 함유', '')
df_allergy['allergy'] = df_allergy['allergy'].str.replace('함유', '')
df_allergy['allergy'] = df_allergy['allergy'].str.replace('함류', '')
df_allergy['allergy'] = df_allergy['allergy'].str.replace('포함', '')
df_allergy['allergy'] = df_allergy['allergy'].str.replace('소고기', '쇠고기')
df_allergy['allergy'] = df_allergy['allergy'].str.replace('괘지', '돼지')
df_allergy['allergy'] = df_allergy['allergy'].str.replace('돼고기', '돼지고기')
df_allergy['allergy'] = df_allergy['allergy'].str.replace('오지엉', '오징어')
df_allergy['allergy'] = df_allergy['allergy'].str.replace('날류','난류')
df_allergy['allergy'] = df_allergy['allergy'].str.replace('유우','우유')
df_allergy['allergy'] = df_allergy['allergy'].str.replace('유유','우유')
df_allergy['allergy'] = df_allergy['allergy'].str.replace('우류','우유')
df_allergy['allergy'] = df_allergy['allergy'].str.replace('탕콩','땅콩')
df_allergy['allergy'] = df_allergy['allergy'].str.replace('대듀','대두')
df_allergy['allergy'] = df_allergy['allergy'].str.replace('토마투','토마토')
df_allergy['allergy'] = df_allergy['allergy'].str.replace('토마도','토마토')
df_allergy['allergy'] = df_allergy['allergy'].str.replace('토미토','토마토')
df_allergy['allergy'] = df_allergy['allergy'].str.replace('이황산류', '아황산류')
df_allergy['allergy'] = df_allergy['allergy'].str.replace('이황산','아황산')
df_allergy['allergy'] = df_allergy['allergy'].str.replace('이산화항','이산화황')
df_allergy['allergy'] = df_allergy['allergy'].str.replace('아산화류', '아황산류')
df_allergy['allergy'] = df_allergy['allergy'].str.replace('아산화','이산화')
df_allergy['allergy'] = df_allergy['allergy'].str.replace('아산화항','이산화황')
df_allergy['allergy'] = df_allergy['allergy'].str.replace('쇠구기','쇠고기')
df_allergy['allergy'] = df_allergy['allergy'].str.replace('·',' ')
df_allergy['allergy'] = df_allergy['allergy'].str.replace('토마','토마토')
df_allergy['allergy'] = df_allergy['allergy'].str.replace('토마토토','토마토')
df_allergy['allergy'] = df_allergy['allergy'].str.replace('게란','계란')
df_allergy.loc[14737, 'allergy'] = '밀, 대두, 계란, 우유, 게, 오징어, 새우, 쇠고기, 조개류'
df_allergy['allergy'] = df_allergy['allergy'].str.replace('닭괴','닭고기')
df_allergy['allergy'] = df_allergy['allergy'].str.replace('쇠소기','쇠고기')
df_allergy['allergy'] = df_allergy['allergy'].str.replace('호도','호두')
df_allergy['allergy'] = df_allergy['allergy'].str.replace('조래규','조개류')
df_allergy['allergy'] = df_allergy['allergy'].str.replace('굴','조개류')
df_allergy['allergy'] = df_allergy['allergy'].str.replace('홍합','조개류')
df_allergy['allergy'] = df_allergy['allergy'].str.replace('바지락','조개류')
df_allergy['allergy'] = df_allergy['allergy'].str.replace('백합','조개류') 
df_allergy['allergy'] = df_allergy['allergy'].str.replace('전복','조개류') 
df_allergy['allergy'] = df_allergy['allergy'].str.replace('등','') 

In [7]:
import re #조개류 통일
pattern = r'조개류\([^\)]+\)'
df_allergy['allergy'] = [re.sub(pattern, '조개류', s) for s in df_allergy['allergy']]
df_allergy['allergy'] = [re.sub(pattern, '조개류', s) for s in df_allergy['allergy']]

In [8]:
import re

pattern = '[^ㄱ-ㅎㅏ-ㅣ가-힣()[\],]+' # 한글, 괄호, 대괄호, 쉼표, 마침표를 제외한 모든 문자
df_allergy['rawmtrl_normalize'] = df_allergy['rawmtrl'].apply(lambda x: re.sub(pattern, ' ', x))

In [ ]:
# rawmtrl 맞춤법 검사 --> 오타가 있음(소백분 같은 경우)
# 우선 정규화를 먼저 진행하고, 그다음에 진행

from hanspell import spell_checker
import json
df_allergy['rawmtrl_checked'] = ""
datas = df_allergy['rawmtrl_normalize'] 
for i, n in enumerate(datas):
    try: 
        input = n.replace("&","").replace("*","").replace("{","").replace("}","").replace("/","") #&가 있으면 xml 문자열에서 에러를 일으킴
        result = spell_checker.check(input)
        result_dict = result.as_dict()
        df_allergy.at[i, 'rawmtrl_checked'] = result_dict['checked']
    except:
        df_allergy.at[i, 'rawmtrl_checked'] = n
        pass

df_allergy.to_csv('data/0508.csv')
#88분 걸림

In [9]:
df_allergy = pd.read_csv('data/0508.csv')

In [10]:
df_allergy['rawmtrl_checked'] = df_allergy['rawmtrl_checked'].str.replace('탈지 대도','탈지대두')
df_allergy['rawmtrl_checked'] = df_allergy['rawmtrl_checked'].str.replace('대도','대두')
df_allergy['rawmtrl_checked'] = df_allergy['rawmtrl_checked'].str.replace('뉴 크림','유크림')
df_allergy['rawmtrl_checked'] = df_allergy['rawmtrl_checked'].str.replace('자란 액','전란액')
df_allergy['rawmtrl_checked'] = df_allergy['rawmtrl_checked'].str.replace('난 황','난황')
df_allergy['rawmtrl_checked'] = df_allergy['rawmtrl_checked'].str.replace('난 각','난각')



In [11]:
# import pandas as pd
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_colwidth', None)

df_allergy['rawmtrl_checked'].sample(10)

10424    정제수, 과당, 이산화탄소, 구연산, 아라비아 검, 합성향료(파인애플 향), 유화제...
2643                                             한우 뼈 추출액 
2265     연육 [외국산(미국, 인도네시아, 베트남 등) 어육, 백설탕, 소르비톨, 산도 조절...
11073    설탕, 망고(페루산) ,망고 퓌레(필리핀산) ,액상과당, 정제수, 망고 농축액 (이...
7324     밀가루 (밀 국산), 버터 (우유 국산), 갈색 설탕, 유정란 (국산), 말차 (국...
7175     카레분 [강황 (인도산), 코리안 뎌(모로코산 캐나다산), 꾸민, 하누 그릭, 펜넬...
8409     정제수, 백설탕, 오렌지 농축과즙(오렌지 과즙으로 미국산),구연산,글루콘산칼슘,아라...
12646    면 쌀면(베트남산 쌀가루, 변성전분, 백설탕, 정제소금, 구아검) 수프류 팟타이 베...
4898     요구르트 크림[식물성유지(싱가포르산), 설탕, 분말이 크림 유크림(호주산) , 혼합...
6960     절임알타리무[알타리무(국내산) , 식염 ] , 양파(국내산) , 고춧가루(국내산) ...
Name: rawmtrl_checked, dtype: object

In [ ]:
# # allergy 맞춤법 검사 오히려 오타가 늘어..ㅋㅋㅋ
# df_allergy['allergy_checked'] = ""
# datas = df_allergy['allergy'] 
# for i, n in enumerate(datas):
#     input = n.replace("&","")
#     result = spell_checker.check(input)
#     result_dict = result.as_dict()
#     df_allergy.at[i, 'allergy_checked'] = result_dict['checked']

## 2. null, 특이사례, 불필요 대상 

In [ ]:
#알러지2칼럼만 사용
#자연어 처리를 통해 rawmtrl 중 알러지 유발 가능 대상을 mtr_allergy에 추가할 수 있음 (1번은 새우, 멸치가 알러지 유발 대상인데, 알러지 정보는 없다고 나와 있음) --> 새로운 컬럼으로 배당, 제공정보 중 제품에서 분석된 것으로 설명
#allergy의 없음, 알수 없음, null에 해당하는 대상들을 자연어 처리로 해결
#분류된 알러지 정보들을 숫자로 변환, 취합
#단, 페닐알라닌과 같이 19개의 번호에 포함되지 않는 경우, 기타(페닐알라닌,.... ) 과 같이 표기할 수 있도록 함
#


# 알러지 딕셔너리, 파일로 저장

In [ ]:
# allergy = ['알류(가금류)', '우유', '메밀', '땅콩', '대두', '밀', '고등어', '게', '새우', '돼지고기', '복숭아', '토마토', '아황산류', '호두', '닭고기', '쇠고기', '오징어', '조개류', '잣']
# allergy_dict = {1: '알류(가금류)', 2: '우유', 3: '메밀', 4: '땅콩', 5: '대두', 6: '밀', 7: '고등어', 8: '게', 9: '새우', 10: '돼지고기', 11: '복숭아', 12: '토마토', 13: '아황산류', 14: '호두', 15: '닭고기', 16: '쇠고기', 17: '오징어', 18: '조개류', 19: '잣'}
# allergy = pd.DataFrame(allergy)
# allergy.reset_index(inplace=True)
# allergy['index']= allergy['index']+1
# allergy.to_csv('data/allergy_table.csv')

# 알러지에 문자열 넣은 경우 확인

In [ ]:
# # ',' 로 구분된 열들을 분리하여 리스트로 변환
# allergy_list = df_allergy['allergy'].str.split(',')

# # 리스트를 바탕으로 새로운 열들을 생성
# new_columns = ['allergy_{}'.format(i+1) for i in range(allergy_list.str.len().max())]
# new_df = df_allergy.assign(**{col: allergy_list.str[i] for i, col in enumerate(new_columns)}) #대충 분리가능한지 검토 

In [ ]:
# new_df[new_df['allergy_20'].notnull()] #아래 있는 것들은 아마도 성분 그냥 때려박은 것으로 보임 

In [ ]:
# new_df.to_csv('data/allergy_search_0504.csv')

In [ ]:
# new_df['allergy_20'].unique()

In [ ]:
# new_df[new_df['allergy_5'] == " 계"]

In [ ]:
# df = pd.read_csv('data/allergy_search_0504.csv')
# df['allergy_1'].unique()

# DF 성분 정규화

In [12]:
df_allergy['rawmtrl_checked'][9033]

'전란액(국내산), 백설탕, 밀가루(미국산), 소르비톨 약, 쇼트닝 (부분 경화유 말레이시아), 쇼트닝, 기타가공품, 식물성 크림, 미강유, 정제수 기타가공품, 식물성유지, 기타 엿, 폴리 글루시톨 시럽, 기타가공품, 난황분, 주정, 폴리텍스트로스, 글리세린, 메이플 시럽, 혼합제제(주정, 젖산, 유화제, 산도 조절제), 정제소금, 과실주, 합성 착향료(카스타 지향, 바닐라 향, 브랜디형), 키토산 분말, 난각 칼슘, 팜 추출물, 키토산 '

In [13]:
df_allergy.sample(5)

,Unnamed: 0,rnum,rawmtrl,allergy,rawmtrl_normalize,rawmtrl_checked
13071,13071,13072,"두유액94%(대두고형분7%이상,대두-수입산),정백당,대두유-수입산,정제염,탄산수소나...",알수없음,"두유액 (대두고형분 이상,대두 수입산),정백당,대두유 수입산,정제염,탄산수소나트륨,...","두어 액 (대두 고형분 이상, 대두 수입품), 정백당, 대두유 수입품, 정제염, 탄..."
6673,6673,6674,계피100%(베트남산),알수없음,계피 (베트남산),계피 (베트남산)
12948,12948,12949,"원유55.00%(국산),정제수,백설탕,유단백질혼합탈지분유(네덜란드산),혼합제제[변성...",우유,"원유 (국산),정제수,백설탕,유단백질혼합탈지분유(네덜란드산),혼합제제[변성전분,덱스...","원유 (국산), 정제수, 백설탕, 유단백질 혼합 탈지 분유(네덜란드산), 혼합제제[..."
11854,11854,11855,"율무흑미밥-정제수,현미(국산),찹쌀,찰흑미,율무쌀, 도라지제육불고기(28.99%)-...",알수없음,"율무흑미밥 정제수,현미(국산),찹쌀,찰흑미,율무쌀, 도라지제육불고기( ) 고추장양념...","율무 흑미밥 정제수, 현미(국산), 찹쌀, 차로 흑미, 율무쌀, 도라지 제육불고기(..."
372,372,373,"국내산 포도 추출액(고형분 5.1%)90%, 유기원당(수입산), 포도농축액(고형분6...",알수없음,"국내산 포도 추출액(고형분 ) , 유기원당(수입산), 포도농축액(고형분 국내산), ...","국내산 포도 추출액(고형분 ) , 유기 원당(수입품), 포도 농축액(고형분 국내산)..."


# 알러지 성분 추출

## allergy 컬럼

In [ ]:
import pandas as pd
from konlpy.tag import Okt

df_allergy['allergy'] = df_allergy['allergy'].str.replace(" ","")
datas = df_allergy['allergy']

# allergy_dict 미리 정의
allergy_dict = {1: ['알류', '계란', '달걀', '난류','메추리알','메추리','추리알','난각','난백','난황','전란'], 
                2: ['우유','탈지분유','분유','유청','카제인','유크림','유청'], 
                3: '메밀', 
                4: ['땅콩', '탕콩'],
                5: ['대두','두부','콩','적두','유부'], 
                6: ['밀','밀가루','소맥','소맥분','밀제품'], 
                7: '고등어', 
                8: '게', 
                9: ['새우'], 
                10: ['돼지고기','돈','돼지','돼고기','돈창', '돈혈', '돈지방', '돈육'],
                11: ['복숭아','황도','백도','천도'], 
                12: ['토마토','토마'],
                13: ['아황산', '아황산류', '아황산나트륨','이산화','이산화황'],
                14: ['호두'], 
                15: ['닭고기', '계육','닭','치킨'], 
                16: ['쇠고기', '우육', '소고기', '소뼈'], 
                17: ['오징어'],
                18: ['조개류', '조개'], 
                19: '잣',
                #20: ['아몬드','헤이즐넛','참깨','피칸'],
                #21: ['페닐알라닌']
                }              

allergy_nums = []
for data in datas:
    data = str(data)
    okt = Okt()
    words = okt.pos(data)
    allergy_columns = []

    for word, tag in words:
        if tag == 'Noun':
            for allergy_key, allergy_value in allergy_dict.items():
                if isinstance(allergy_value, list):
                    if word in allergy_value:
                        allergy_columns.append(allergy_key)
                        break
                else:
                    if word == allergy_value:
                        allergy_columns.append(allergy_key)
                        break
        
    allergy_num = ','.join(str(x) for x in allergy_columns) # 문자열로 변환
    allergy_nums.append(allergy_num)

df_allergy['allergy_num'] = allergy_nums # DataFrame에 열 추가

## rawmtrl 컬럼

In [ ]:
import pandas as pd
from konlpy.tag import Okt

datas = df_allergy['rawmtrl']

# allergy_dict 미리 정의
allergy_dict = {1: ['알류', '계란', '달걀', '난류','메추리알','메추리','추리알','난각','날류','난백','난황','난각','게란','전란','흰자','노른자'], 
                2: ['우유','유우', '탈지분유','유유','분유','유청','카제인','우류','가공버터','유크림','크림','유청','치즈','원유'], 
                3: '메밀', 
                4: ['땅콩', '탕콩'],
                5: ['대두','두부','콩','유부','된장','대두유','대두단백'], 
                6: ['밀','밀가루','소맥','소맥분', '밀제품'], 
                7: ['고등어'], 
                8: ['게'], 
                9: ['새우'], 
                10: ['돼지','돈','돼지고기','괘지','괘지고기','돼고기','돈창', '돈혈', '돈지방','괘','젤라틴'],
                11: ['복숭아','황도','백도','천도'], 
                12: ['토마토', '토마투','토마도','토마','토미토'],
                13: ['아황산', '아황산나트륨', '이황산', '이산화', '아산화', '이산화황','아산화황','이산화항','이황산류'],
                14: ['호두','호도'], 
                15: ['닭', '닭고기', '계육','닭괴','치킨','닭다리'], 
                16: ['쇠고기', '우육', '소고기', '소뼈','쇠구기', '한우', '우정육', '소정육','소뼈','소잡뼈','우골'], 
                17: ['오징어', '오지엉','오지','지엉'],
                18: ['조개류','굴','홍합','조개','백합','바지락','전복'], 
                19: '잣',
                #20: ['아몬드','헤이즐넛','참깨','피칸'],
                #21: ['페닐알라닌']
                }
                
allergy_nums = []
for data in datas:
    data = str(data)
    okt = Okt()
    words = okt.pos(data)
    allergy_columns = []

    for word, tag in words:
        if tag == 'Noun':
            for allergy_key, allergy_value in allergy_dict.items():
                if isinstance(allergy_value, list):
                    if word in allergy_value:
                        allergy_columns.append(allergy_key)
                        break
                else:
                    if word == allergy_value:
                        allergy_columns.append(allergy_key)
                        break
        
    allergy_num = ','.join(str(x) for x in allergy_columns) # 문자열로 변환
    allergy_nums.append(allergy_num)

df_allergy['rawmtrl_num'] = allergy_nums # DataFrame에 열 추가

In [ ]:
df_allergy.head()

## rawmtrl_checked

In [ ]:
import pandas as pd
from konlpy.tag import Okt

df_allergy['rawmtrl_checked'] = df_allergy['rawmtrl_checked'].str.replace(" ","")
datas = df_allergy['rawmtrl_checked']

# allergy_dict 미리 정의
allergy_dict = {1: ['알류', '계란', '달걀', '난류','메추리알','메추리','추리알','난각','날류','난백','난황','난각','게란','전란','흰자','노른자'], 
                2: ['우유','유우', '탈지분유','유유','분유','유청','카제인','우류','가공버터','유크림','크림','유청','치즈','원유'], 
                3: '메밀', 
                4: ['땅콩', '탕콩'],
                5: ['대두','두부','콩','유부','된장','대두유','대두단백'], 
                6: ['밀','밀가루','소맥','소맥분', '밀제품'], 
                7: ['고등어'], 
                8: ['게'], 
                9: ['새우'], 
                10: ['돼지','돈','돼지고기','괘지','괘지고기','돼고기','돈창', '돈혈', '돈지방','괘','젤라틴'],
                11: ['복숭아','황도','백도','천도'], 
                12: ['토마토', '토마투','토마도','토마','토미토'],
                13: ['아황산', '아황산나트륨', '이황산', '이산화', '아산화', '이산화황','아산화황','이산화항','이황산류'],
                14: ['호두','호도'], 
                15: ['닭', '닭고기', '계육','닭괴','치킨','닭다리'], 
                16: ['쇠고기', '우육', '소고기', '소뼈','쇠구기', '한우', '우정육', '소정육','소뼈','소잡뼈','우골'], 
                17: ['오징어', '오지엉','오지','지엉'],
                18: ['조개류','굴','홍합','조개','백합','바지락','전복'], 
                19: '잣',
                #20: ['아몬드','헤이즐넛','참깨','피칸'],
                #21: ['페닐알라닌']
                }
                
allergy_nums = []
for data in datas:
    data = str(data)
    okt = Okt()
    words = okt.pos(data)
    allergy_columns = []

    for word, tag in words:
        if tag == 'Noun':
            for allergy_key, allergy_value in allergy_dict.items():
                if isinstance(allergy_value, list):
                    if word in allergy_value:
                        allergy_columns.append(allergy_key)
                        break
                else:
                    if word == allergy_value:
                        allergy_columns.append(allergy_key)
                        break
        
    allergy_num = ','.join(str(x) for x in allergy_columns) # 문자열로 변환
    allergy_nums.append(allergy_num)

df_allergy['rawmtrl_num2'] = allergy_nums # DataFrame에 열 추가

In [19]:
df_allergy.fillna("")

,Unnamed: 0,rnum,rawmtrl,allergy,rawmtrl_normalize,rawmtrl_checked,tokens1
0,0,1,"찹쌀,김,참깨,옥수수기름(옥배유),양파,무,대파,천일염,마늘,새우,멸치,다시마,건표...",없음,"찹쌀,김,참깨,옥수수기름(옥배유),양파,무,대파,천일염,마늘,새우,멸치,다시마,건표...","찹쌀, 김, 참깨, 옥수수기름(옥배 유), 양파, 무, 대파, 천일염, 마늘, 새우...","[찹쌀, 김, 참깨, 옥수수기름, 옥배유, 양파, 무, 대파, 천일염, 마늘, 새우..."
1,1,2,"찹쌀,김,참깨,옥수수기름(옥배유),아몬드,양파,무,천일염,대파,마늘,새우,멸치,다시...",아몬드,"찹쌀,김,참깨,옥수수기름(옥배유),아몬드,양파,무,천일염,대파,마늘,새우,멸치,다시...","찹쌀, 김, 참깨, 옥수수기름(옥배 유), 아몬드, 양파, 무, 천일염, 대파, 마...","[찹쌀, 김, 참깨, 옥수수기름, 옥배유, 아몬드, 양파, 무, 천일염, 대파, 마..."
2,2,3,"찹쌀,김,참깨,옥수수기름(옥배유),양파,무,대파,천일염,마늘,새우,멸치,다시마,건표...",없음,"찹쌀,김,참깨,옥수수기름(옥배유),양파,무,대파,천일염,마늘,새우,멸치,다시마,건표...","찹쌀, 김, 참깨, 옥수수기름(옥배 유), 양파, 무, 대파, 천일염, 마늘, 새우...","[찹쌀, 김, 참깨, 옥수수기름, 옥배유, 양파, 무, 대파, 천일염, 마늘, 새우..."
3,3,4,군고구마,없음,군고구마,군고구마,[군고구마]
4,4,5,군고구마,없음,군고구마,군고구마,[군고구마]
...,...,...,...,...,...,...,...
15087,15087,15088,"된장63.15%[대두{수입산(미국산,중국산,캐나다산)},소맥분(밀:미국산,호주산),...","대두,밀","된장 [대두 수입산(미국산,중국산,캐나다산) ,소맥분(밀 미국산,호주산),정제소금,...","된장 [대두 수입품(미국산, 중국산, 캐나다산) ,소맥분(밀 미국산, 호주산), 정...","[된장, 대두, 수입산, 미국산, 중국산, 캐나다산, 소맥분, 밀, 미국산, 호주산..."
15088,15088,15089,"물엿,소맥분(밀:미국산,호주산),혼합양념(중국산)[고춧가루6.12%,(고추:중국산)...",밀,"물엿,소맥분(밀 미국산,호주산),혼합양념(중국산)[고춧가루 ,(고추 중국산),정제소...","물엿, 소맥분(밀 미국산, 호주산), 혼합양념(중국산)[고춧가루 ,(고추 중국산),...","[물엿, 소맥분, 밀, 미국산, 호주산, 혼합양념, 중국산, 고춧가루, 고추, 중국..."
15089,15089,15090,"탈지대두[수입산(인도산,미국산,중국산)],정제수,양조간장원액(대만산)[탈지대두,소맥...","대두,밀","탈지대두[수입산(인도산,미국산,중국산)],정제수,양조간장원액(대만산)[탈지대두,소맥...","탈지대두[수입품(인도산, 미국산, 중국산)], 정제수, 양조간장 원액(대만산)[탈지...","[탈지대두, 수입산, 인도산, 미국산, 중국산, 정제수, 양조간장원액, 대만산, 탈..."
15090,15090,15091,"정제수,아미노산액[탈지대두:외국산(인도산,미국산,중국산등)],정제소금(국산),액상과...","대두,밀","정제수,아미노산액[탈지대두 외국산(인도산,미국산,중국산등)],정제소금(국산),액상과...","정제수,아미노산액[탈지대두 외국산(인도산, 미국산, 중국산 등)], 정제소금(국산)...","[정제수, 아미노산액, 탈지대두, 외국산, 인도산, 미국산, 중국산등, 정제소금, ..."


In [21]:
# import re
# from konlpy.tag import Okt

# # 한글 정규화 함수
# def normalize_text(text):
#     # 중복 공백 제거
#     text = re.sub(r'\s+', ' ', text)
#     # 한글 정규화
#     text = re.sub(r'[^가-힣\s]', '', text)
#     return text.strip()

# # Mecab 토크나이저 초기화
# tokenizer = Okt()

# # 문장 정규화 후 토큰화
# df_allergy['tokens1'] = df_allergy['rawmtrl'].apply(normalize_text).apply(tokenizer.morphs)
# df_allergy['tokens2'] = df_allergy['rawmtrl_checked'].apply(normalize_text).apply(tokenizer.morphs)

# # 결과 확인
# print(df_allergy.head())

import re
from soynlp.normalizer import normalize
from soynlp.tokenizer import MaxScoreTokenizer

#한글 정규화 함수
def normalize_text(text):
    # 중복 공백 제거
    text = re.sub(r'\s+', ' ', text)
    # 한글 정규화
    text = normalize(text)
    return text.strip()

#soynlp 토크나이저 초기화
tokenizer = MaxScoreTokenizer()

#문장 정규화 후 토큰화
df_allergy['tokens1'] = df_allergy['rawmtrl'].apply(normalize_text).apply(tokenizer.tokenize)
#df_allergy['tokens2'] = df_allergy['rawmtrl_checked'].apply(normalize_text).apply(tokenizer.tokenize)

#결과 확인
print(df_allergy.head())

   Unnamed: 0  rnum                                            rawmtrl  \
0           0     1  찹쌀,김,참깨,옥수수기름(옥배유),양파,무,대파,천일염,마늘,새우,멸치,다시마,건표...   
1           1     2  찹쌀,김,참깨,옥수수기름(옥배유),아몬드,양파,무,천일염,대파,마늘,새우,멸치,다시...   
2           2     3  찹쌀,김,참깨,옥수수기름(옥배유),양파,무,대파,천일염,마늘,새우,멸치,다시마,건표...   
3           3     4                                               군고구마   
4           4     5                                               군고구마   

  allergy                                  rawmtrl_normalize  \
0      없음  찹쌀,김,참깨,옥수수기름(옥배유),양파,무,대파,천일염,마늘,새우,멸치,다시마,건표...   
1     아몬드  찹쌀,김,참깨,옥수수기름(옥배유),아몬드,양파,무,천일염,대파,마늘,새우,멸치,다시...   
2      없음  찹쌀,김,참깨,옥수수기름(옥배유),양파,무,대파,천일염,마늘,새우,멸치,다시마,건표...   
3      없음                                               군고구마   
4      없음                                               군고구마   

                                     rawmtrl_checked  \
0  찹쌀, 김, 참깨, 옥수수기름(옥배 유), 양파, 무, 대파, 천일염, 마늘, 새우...   
1  찹쌀, 김, 참깨, 옥수수기름(옥배 유), 아몬드, 양파, 무, 천일염, 대파, 마...   
2 

In [22]:
df_allergy.head()

,Unnamed: 0,rnum,rawmtrl,allergy,rawmtrl_normalize,rawmtrl_checked,tokens1
0,0,1,"찹쌀,김,참깨,옥수수기름(옥배유),양파,무,대파,천일염,마늘,새우,멸치,다시마,건표...",없음,"찹쌀,김,참깨,옥수수기름(옥배유),양파,무,대파,천일염,마늘,새우,멸치,다시마,건표...","찹쌀, 김, 참깨, 옥수수기름(옥배 유), 양파, 무, 대파, 천일염, 마늘, 새우...","[찹쌀, 김, 참깨, 옥수수기름, 옥배유, 양파, 무, 대파, 천일염, 마늘, 새우..."
1,1,2,"찹쌀,김,참깨,옥수수기름(옥배유),아몬드,양파,무,천일염,대파,마늘,새우,멸치,다시...",아몬드,"찹쌀,김,참깨,옥수수기름(옥배유),아몬드,양파,무,천일염,대파,마늘,새우,멸치,다시...","찹쌀, 김, 참깨, 옥수수기름(옥배 유), 아몬드, 양파, 무, 천일염, 대파, 마...","[찹쌀, 김, 참깨, 옥수수기름, 옥배유, 아몬드, 양파, 무, 천일염, 대파, 마..."
2,2,3,"찹쌀,김,참깨,옥수수기름(옥배유),양파,무,대파,천일염,마늘,새우,멸치,다시마,건표...",없음,"찹쌀,김,참깨,옥수수기름(옥배유),양파,무,대파,천일염,마늘,새우,멸치,다시마,건표...","찹쌀, 김, 참깨, 옥수수기름(옥배 유), 양파, 무, 대파, 천일염, 마늘, 새우...","[찹쌀, 김, 참깨, 옥수수기름, 옥배유, 양파, 무, 대파, 천일염, 마늘, 새우..."
3,3,4,군고구마,없음,군고구마,군고구마,[군고구마]
4,4,5,군고구마,없음,군고구마,군고구마,[군고구마]


In [23]:
import pandas as pd
from konlpy.tag import Okt

datas = df_allergy['tokens1']

# allergy_dict 미리 정의
allergy_dict = {1: ['알류', '계란', '달걀', '난류','메추리알','메추리','추리알','난각','날류','난백','난황','난각','게란','전란','흰자','노른자'], 
                2: ['우유','유우', '탈지분유','유유','분유','유청','카제인','우류','가공버터','유크림','크림','유청','치즈','원유'], 
                3: '메밀', 
                4: ['땅콩', '탕콩'],
                5: ['대두','두부','콩','유부','된장','대두유','대두단백'], 
                6: ['밀','밀가루','소맥','소맥분', '밀제품'], 
                7: ['고등어'], 
                8: ['게'], 
                9: ['새우'], 
                10: ['돼지','돈','돼지고기','괘지','괘지고기','돼고기','돈창', '돈혈', '돈지방','괘','젤라틴'],
                11: ['복숭아','황도','백도','천도'], 
                12: ['토마토', '토마투','토마도','토마','토미토'],
                13: ['아황산', '아황산나트륨', '이황산', '이산화', '아산화', '이산화황','아산화황','이산화항','이황산류'],
                14: ['호두','호도'], 
                15: ['닭', '닭고기', '계육','닭괴','치킨','닭다리'], 
                16: ['쇠고기', '우육', '소고기', '소뼈','쇠구기', '한우', '우정육', '소정육','소뼈','소잡뼈','우골'], 
                17: ['오징어', '오지엉','오지','지엉'],
                18: ['조개류','굴','홍합','조개','백합','바지락','전복'], 
                19: '잣',
                #20: ['아몬드','헤이즐넛','참깨','피칸'],
                #21: ['페닐알라닌']
                }
                
allergy_nums = []
for data in datas:
    data = str(data)
    okt = Okt()
    words = okt.pos(data)
    allergy_columns = []

    for word, tag in words:
        if tag == 'Noun':
            for allergy_key, allergy_value in allergy_dict.items():
                if isinstance(allergy_value, list):
                    if word in allergy_value:
                        allergy_columns.append(allergy_key)
                        break
                else:
                    if word == allergy_value:
                        allergy_columns.append(allergy_key)
                        break
        
    allergy_num = ','.join(str(x) for x in allergy_columns) # 문자열로 변환
    allergy_nums.append(allergy_num)

df_allergy['rawmtrl_num3'] = allergy_nums # DataFrame에 열 추가

In [ ]:
import pandas as pd
from konlpy.tag import Okt

datas = df_allergy['tokens2']

# allergy_dict 미리 정의
allergy_dict = {1: ['알류', '계란', '달걀', '난류','메추리알','메추리','추리알','난각','날류','난백','난황','난각','게란','전란','흰자','노른자'], 
                2: ['우유','유우', '탈지분유','유유','분유','유청','카제인','우류','가공버터','유크림','크림','유청','치즈','원유'], 
                3: '메밀', 
                4: ['땅콩', '탕콩'],
                5: ['대두','두부','콩','유부','된장','대두유','대두단백'], 
                6: ['밀','밀가루','소맥','소맥분', '밀제품'], 
                7: ['고등어'], 
                8: ['게'], 
                9: ['새우'], 
                10: ['돼지','돈','돼지고기','괘지','괘지고기','돼고기','돈창', '돈혈', '돈지방','괘','젤라틴'],
                11: ['복숭아','황도','백도','천도'], 
                12: ['토마토', '토마투','토마도','토마','토미토'],
                13: ['아황산', '아황산나트륨', '이황산', '이산화', '아산화', '이산화황','아산화황','이산화항','이황산류'],
                14: ['호두','호도'], 
                15: ['닭', '닭고기', '계육','닭괴','치킨','닭다리'], 
                16: ['쇠고기', '우육', '소고기', '소뼈','쇠구기', '한우', '우정육', '소정육','소뼈','소잡뼈','우골'], 
                17: ['오징어', '오지엉','오지','지엉'],
                18: ['조개류','굴','홍합','조개','백합','바지락','전복'], 
                19: '잣',
                #20: ['아몬드','헤이즐넛','참깨','피칸'],
                #21: ['페닐알라닌']
                }
                
allergy_nums = []
for data in datas:
    data = str(data)
    okt = Okt()
    words = okt.pos(data)
    allergy_columns = []

    for word, tag in words:
        if tag == 'Noun':
            for allergy_key, allergy_value in allergy_dict.items():
                if isinstance(allergy_value, list):
                    if word in allergy_value:
                        allergy_columns.append(allergy_key)
                        break
                else:
                    if word == allergy_value:
                        allergy_columns.append(allergy_key)
                        break
        
    allergy_num = ','.join(str(x) for x in allergy_columns) # 문자열로 변환
    allergy_nums.append(allergy_num)

df_allergy['rawmtrl_num4'] = allergy_nums # DataFrame에 열 추가

In [ ]:
df_allergy['num_sum'] = df_allergy['allergy_num']+','+df_allergy['rawmtrl_num']+','+df_allergy['rawmtrl_num2']+','+df_allergy['rawmtrl_num3']+','+df_allergy['rawmtrl_num4']
df_allergy['num_sum']

In [24]:
def remove_duplicates(str_nums):
    nums = set(str_nums.split(','))
    nums.discard('')
    return ','.join(sorted(nums))

#df_allergy['num_sum'] = df_allergy['num_sum'].apply(remove_duplicates)
df_allergy['rawmtrl_num3'] = df_allergy['rawmtrl_num3'].apply(remove_duplicates)

In [25]:

allergy_dict = {1: '알류', 2: '우유', 3: '메밀', 4: '땅콩', 5: '콩', 6: '밀', 7: '고등어', 8: '게', 9: '새우', 10: '돼지고기', 11: '복숭아', 12: '토마토', 13: '아황산류', 14: '호두', 15: '닭고기', 16: '쇠고기', 17: '오징어', 18: '조개류', 19: '잣'}  #, 20:'견과류', 21:'페닐알라닌'} 삭제

def convert_to_allergy(num_str):
    num_list = num_str.split(',')
    allergy_list = []
    for num in num_list:
        if num.strip() == '':
            continue
        allergy_list.append(allergy_dict[int(num)])
    return ','.join(allergy_list)

#df_allergy['new_allergy'] = df_allergy['num_sum'].apply(convert_to_allergy)
df_allergy['new_allergy'] = df_allergy['rawmtrl_num3'].apply(convert_to_allergy)

In [27]:
df_allergy.sample(20)

,Unnamed: 0,rnum,rawmtrl,allergy,rawmtrl_normalize,rawmtrl_checked,tokens1,rawmtrl_num3,new_allergy
12313,12313,12314,"가공 초콜릿[설탕,가공유지(팜핵경화에스테르화유(말레이시아산),팜핵에스테르화유(말레이...","우유,대두","가공 초콜릿[설탕,가공유지(팜핵경화에스테르화유(말레이시아산),팜핵에스테르화유(말레이...","가공 초콜릿[설탕,가공유지(팜핵경화에스테르화유(말레이시아산),팜핵에스테르화유(말레이...","[가공, 초콜릿, 설탕, 가공유지, 팜핵경화에스테르화유, 말레이시아산, 팜핵에스테르...",2,우유
8204,8204,8205,"돼지고기(국산)45.29%,닭고기(국산/기계발골육)21.65%,정제수,전분(옥수수/...","돼지고기,닭고기,대두,밀,알류,우유","돼지고기(국산) ,닭고기(국산 기계발골육) ,정제수,전분(옥수수 외국산 러시아,세르...","돼지고기(국산) ,닭고기(국산 기계 팔 골육) ,정제수, 전분(옥수수 외국산 러시아...","[돼지고기, 국산, 닭고기, 국산, 기계발골육, 정제수, 전분, 옥수수, 외국산, ...","10,15,5","돼지고기,닭고기,콩"
5984,5984,5985,"된장58.12%[대두(수입산),소맥분(밀:미국,호주산),정제소금,종국],물엿,소맥분...","밀, 대두","된장 [대두(수입산),소맥분(밀 미국,호주산),정제소금,종국],물엿,소맥분(밀 미국...","된장 [대두(수입품), 소맥분(밀 미국, 호주산), 정제소금, 종국], 물엿, 소맥...","[된장, 대두, 수입산, 소맥분, 밀, 미국, 호주산, 정제소금, 종국, 물엿, 소...","5,6","콩,밀"
8281,8281,8282,"두유66.7%(대두고형분6%,대두:미국산),정제수,백설탕,바나나농축액(바나나과즙으로...","대두,우유","두유 (대두고형분 ,대두 미국산),정제수,백설탕,바나나농축액(바나나과즙으로 ,미국산...","두유 (대두 고형분, 대두 미국산), 정제수, 백설탕, 바나나 농축액(바나나 과즙으...","[두유, 대두고형분, 대두, 미국산, 정제수, 백설탕, 바나나농축액, 바나나과즙으로...","2,5","우유,콩"
13893,13893,13894,"콩기름 99.99%(콩,수입산), 규소수지",알수없음,"콩기름 (콩,수입산), 규소수지","콩기름 (콩, 수입품), 규소수지","[콩기름, 콩, 수입산, 규소수지]",5,콩
9903,9903,9904,"돼지고기92.90%(국산),대두단백(중국산),설탕,정제소금(국산),L-글루타민산나트...","돼지고기,대두,쇠고기,밀,계란","돼지고기 (국산),대두단백(중국산),설탕,정제소금(국산), 글루타민산나트륨(향미증진...","돼지고기 (국산), 대두 단백(중국산), 설탕, 정제소금(국산), 글루타민산 나트륨...","[돼지고기, 국산, 대두단백, 중국산, 설탕, 정제소금, 국산, 글루타민산나트륨, ...","1,10,5","알류,돼지고기,콩"
10626,10626,10627,"순부부-대두[외국산(미국.태나다,중국등)],두부응고제/바지락육수-정제수,바지락밑국물...","돼지고기,조개류,새우,게,대두,밀","순부부 대두[외국산(미국 태나다,중국등)],두부응고제 바지락육수 정제수,바지락밑국물...",NaN,"[순부부, 대두, 외국산, 미국, 태나다, 중국등, 두부응고제, 바지락육수, 정제수...","18,5,6,8,9","조개류,콩,밀,게,새우"
11160,11160,11161,"밀가루(밀/미국산,호주산),물엿,백설탕,채종유[유채씨유/호주산,규소수지],물엿,물엿...",알수없음,"밀가루(밀 미국산,호주산),물엿,백설탕,채종유[유채씨유 호주산,규소수지],물엿,물엿...","밀가루(밀 미국산, 호주산), 물엿, 백설탕, 채종유[유채씨유 호주산, 규소수지],...","[밀가루, 밀, 미국산, 호주산, 물엿, 백설탕, 채종유, 유채씨유, 호주산, 규소...",6,밀
6889,6889,6890,"정제수, 배퓨레 12%{배(국산)99.95%, 비타민C 0.05%}, 사과퓨레 12...",알수없음,"정제수, 배퓨레 배(국산) , 비타민 , 사과퓨레 사과(국산) , 비타민 , 폴리덱...","정제수, 배퓨레 배(국산) , 비타민, 사과 퓌레 사과(국산) , 비타민, 폴리텍스...","[정제수, 배퓨레, 배, 국산, 비타민, 사과퓨레, 사과, 국산, 비타민, 폴리덱스...",,
11991,11991,11992,"정제수,유기쌀18%(국산),채소상탕[무(국산),양파(국산),당근(국산),대파,양배추...","쇠고기, 닭고기, 대두, 밀, 호두","정제수,유기쌀 (국산),채소상탕[무(국산),양파(국산),당근(국산),대파,양배추,쌈...","정제수, 유기쌀 (국산), 채소 상탕[무(국산), 양파(국산), 당근(국산), 대파...","[정제수, 유기쌀, 국산, 채소상탕, 무, 국산, 양파, 국산, 당근, 국산, 대파...","14,15,16,5","호두,닭고기,쇠고기,콩"


In [ ]:
for index, row in df_allergy.iterrows():
    if row['new_allergy'] == "":
        df_allergy.loc[index, 'sentence'] = row['rawmtrl'] + '의 알레르기 유발성분은 없다.'
    else:
        df_allergy.loc[index, 'sentence'] = row['rawmtrl'] + '의 알레르기 유발성분은 ' + row['new_allergy'] + '이다.'


In [ ]:
# import re

# # 정규표현식 패턴: 한글 외의 문자 1개 이상을 모두 제거
# pattern = '[^ㄱ-ㅎㅏ-ㅣ가-힣]+'

# # 'sentence' 컬럼의 모든 값을 정규화하여 'normalized' 컬럼에 저장
# df_allergy['normalized'] = df_allergy['sentence'].apply(lambda x: re.sub(pattern, '', x))

In [ ]:
import re
from konlpy.tag import Okt

# 한글 정규화 함수
def normalize_text(text):
    # 중복 공백 제거
    text = re.sub(r'\s+', ' ', text)
    # 한글 정규화
    text = re.sub(r'[^가-힣\s]', '', text)
    return text.strip()

# Mecab 토크나이저 초기화
tokenizer = Okt()

# 문장 정규화 후 토큰화
df_allergy['tokens'] = df_allergy['sentence'].apply(normalize_text).apply(tokenizer.morphs)

# 결과 확인
print(df_allergy.head())

In [ ]:
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split

In [ ]:
df_allergy

In [ ]:
# 학습 데이터와 검증 데이터로 분리
train_data, test_data = train_test_split(df_allergy['tokens'], test_size=0.2, random_state=42)

# Word2Vec 모델 학습
model = Word2Vec(sentences=train_data, vector_size=150, window=5, min_count=5, workers=12, sg=1)

# 모델 저장
model.save('allergy_word2vec.model')


In [ ]:
# # 검증 데이터로 평가
# accuracy = model.wv.evaluate_word_pairs('test_word_pairs.txt', delimiter='\t')
# print('Accuracy:', accuracy)

# # 검증 데이터로 일부 추가 학습
# model.train(test_data, total_examples=len(test_data), epochs=10)

# # 검증 데이터로 평가
# accuracy = model.wv.evaluate_word_pairs('test_word_pairs.txt', delimiter='\t')
# print('Accuracy after fine-tuning:', accuracy)

In [ ]:
!pip install seaborn

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE

# 모델의 단어 벡터를 추출
word_vectors = model.wv

# 가장 유사한 단어 찾기
similar_words = {search_term: [item[0] for item in word_vectors.most_similar([search_term], topn=5)]
                  for search_term in ['소고기', '돼지고기','닭고기','대두','콩', '새우','게','아황산']}

# 찾은 단어들에 대한 단어 벡터 가져오기
words = sum([[k] + v for k, v in similar_words.items()], [])
wvs = model.wv[words]

# t-SNE를 사용하여 단어 벡터 시각화
tsne = TSNE(n_components=2, random_state=0, n_iter=10000, perplexity=5)
np.set_printoptions(suppress=True)
T = tsne.fit_transform(wvs)
labels = words

plt.figure(figsize=(12, 6))
sns.scatterplot(x=T[:, 0], y=T[:, 1], hue=[label.split(':')[0] for label in labels])
for label, x, y in zip(labels, T[:, 0], T[:, 1]):
    plt.annotate(label, xy=(x+1, y+1), xytext=(0, 0), textcoords='offset points')
    plt.xlim(T[:, 0].min()-50, T[:, 0].max()+50)
    plt.ylim(T[:, 1].min()-50, T[:, 1].max()+50)
plt.show()
